In [42]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
import matplotlib.pyplot as plt
from bayes_opt import BayesianOptimization

In [43]:
# Step 1: Preprocess the data
# Load the dataset
data = pd.read_csv('data.csv')

In [44]:
# Parse dates and set it as the index
data['Date'] = pd.to_datetime(data['Date'])
data.set_index('Date', inplace=True)

In [45]:
# Normalize the price values
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(data['Price'].values.reshape(-1, 1))

In [46]:
# Split the dataset into training and testing sets
train_size = int(len(scaled_data) * 0.8)
train_data = scaled_data[:train_size]
test_data = scaled_data[train_size:]

In [47]:
# Generate input sequences and corresponding labels
def generate_sequences(data, sequence_length):
    X = []
    y = []
    for i in range(len(data) - sequence_length):
        X.append(data[i:i+sequence_length])
        y.append(data[i+sequence_length])
    return np.array(X), np.array(y)

In [48]:
sequence_length = 100

In [49]:
X_train, y_train = generate_sequences(train_data, sequence_length)
X_test, y_test = generate_sequences(test_data, sequence_length)

In [50]:
# Step 2: Define the LSTM model
def create_model(optimizer='adam', loss='mean_squared_error', batch_size=16, units=128):
    model = Sequential()
    model.add(LSTM(units, input_shape=(sequence_length, 1)))
    model.add(Dense(1))
    model.compile(loss=loss, optimizer=optimizer, metrics=['accuracy'])
    return model

In [51]:
# Step 3: Define the Bayesian optimization objective
def objective(batch_size, units):
    batch_size = int(batch_size)
    units = int(units)
    model = create_model(optimizer='adam', loss='mean_squared_error', batch_size=batch_size, units=units)
    model.fit(X_train, y_train, epochs=10, batch_size=batch_size, validation_data=(X_test, y_test))
    loss = model.evaluate(X_test, y_test, verbose=0)
    return -loss

In [52]:
# Step 4: Create the Bayesian optimization object
optimizer = BayesianOptimization(
    f=objective,
    pbounds={
        'batch_size': (4, 16),
        'units': (128, 256)
    },
    verbose=1
)

In [ ]:
# Step 5: Run the Bayesian optimization
optimizer.maximize(n_iter=100)

In [ ]:
# Step 6: Print the best parameters
print(optimizer.max)